# Word Detection Models

# 1. PDF Pre-processing

# 1.1. Working on converting pdfs to images

In [1]:
import cv2
import pdf2image
from matplotlib import pyplot as plt

In [2]:
import os

pdfs = os.listdir('pdfs')

In [3]:
pdfs

['gosast e_fasle-1.pdf', '.file', 'ubuntu-book.pdf', 'justforfun_persian.pdf']

## 1.2. Making an array of images from PDF files

In [7]:
from pdf2image import convert_from_path

images = []

for pdf in pdfs:
    try:
    pages = convert_from_path(f'pdfs/{pdf}')
    images.extend(pages)
    except Exception as e:
        print(e)

IndentationError: expected an indented block (<ipython-input-7-b6639b0f9525>, line 7)